# 3. Pré-processamento & Análise de dados

- **Pré-processamento de dados:** Eliminamos os duplicados, substituímos quaisquer valores infinitos (positivos ou negativos) por NaN (not a number) e preenchemos os valores em falta com valores medianos. Como temos um conjunto de dados muito grande, o uso inicial de memória é bastante alto, levando a falhas na sessão. Posteriormente, resolvemos o problema fazendo down casting de tipos de dados com base nos valores mínimo e máximo disponíveis. Tentamos reduzir o uso de memória que é útil para o nosso modelo.
- **Análise de dados:** Agrupámos ataques semelhantes para analisar o conjunto de dados e identificar padrões nos diferentes tipos de ataques. Recolhemos uma amostra da população (20%). Posteriormente, efectuámos uma análise de dados que consistiu em traçar vários tipos de gráficos, matrizes de correlação, etc., para ver as relações entre caraterísticas, tipos de ataques presentes no conjunto de dados, etc. Na nossa análise, verificámos que há um bom número de caraterísticas que estão fortemente, ou mesmo diretamente, correlacionadas com outras caraterísticas (tanto positivas como negativas). Este é um problema que introduz a multicolinearidade, que pode ter um grande impacto nos modelos de aprendizagem automática que iremos desenvolver mais tarde.

  Além disso, como o conjunto de dados é bastante grande e tem mais de 70 caraterísticas, seria extremamente difícil treinar modelos utilizando recursos limitados. Para ultrapassar este problema, utilizámos a PCA (análise de componentes principais) para reduzir as dimensões. Jogámos com o número de componentes para ver quanta informação conseguimos preservar. Monitorizámos o '*explained_variance_ratio_*' para nos certificarmos de que retemos a maior parte das informações. No entanto, foi um pouco difícil reduzir as dimensões e, ao mesmo tempo, preservar as informações para treinar os modelos. Executámos o StandardScaler antes de executar o Incremental PCA.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
sns.set(style='darkgrid')
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv('datasets/CICIDS2017/CIC_IDS_2017_pre_Limpo.csv')

In [3]:
# Calculando o uso de memória inicial do DataFrame
old_memory_usage = data.memory_usage().sum() / 1024 ** 2  # Converte de bytes para MB
print(f'Uso de memória inicial: {old_memory_usage:.2f} MB')

# Loop através de todas as colunas do DataFrame
for col in data.columns:
    col_type = data[col].dtype  # Obtém o tipo de dado da coluna
    if col_type != object:  # Verifica se a coluna não é do tipo 'object' (não será alterada, como strings)
        
        c_min = data[col].min()  # Valor mínimo da coluna
        c_max = data[col].max()  # Valor máximo da coluna

        # Verifica se a coluna é do tipo float e se os valores estão dentro dos limites do float32
        if str(col_type).find('float') >= 0 and c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
            data[col] = data[col].astype(np.float32)  # Converte a coluna para float32, economizando memória

        # Verifica se a coluna é do tipo int e se os valores estão dentro dos limites do int32
        elif str(col_type).find('int') >= 0 and c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
            data[col] = data[col].astype(np.int32)  # Converte a coluna para int32, economizando memória

# Calculando o uso de memória após a conversão dos tipos de dados
new_memory_usage = data.memory_usage().sum() / 1024 ** 2  # Converte de bytes para MB
print(f"Uso de memória final: {new_memory_usage:.2f} MB")

Uso de memória inicial: 1520.28 MB
Uso de memória final: 789.01 MB


In [4]:
# Calculando a redução percentual no uso de memória
print(f'Redução no uso de memória: {1 - (new_memory_usage / old_memory_usage):.2%}')

Redução no uso de memória: 48.10%


In [5]:
# Contando o número de valores únicos em cada coluna
num_unique = data.nunique()

# Encontrando as colunas com apenas um valor único
one_variable = num_unique[num_unique == 1]

# Encontrando as colunas com mais de um valor único (que serão mantidas)
not_one_variable = num_unique[num_unique > 1].index

# Armazenando as colunas que foram descartadas
dropped_cols = one_variable.index

# Mantendo apenas as colunas com mais de um valor único
data = data[not_one_variable]

# Exibindo as colunas que foram descartadas
print('Colunas descartadas:')
print(dropped_cols)

Colunas descartadas:
Index(['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk',
       'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk',
       'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'],
      dtype='object')


- Para melhorar o desempenho e reduzir o risco de erros relacionados com a memória (principalmente falhas de sessão), reduzimos os valores flutuantes e inteiros com base na presença dos valores mínimo e máximo e reduzimos a utilização de memória em 47,5%.

- As colunas com desvio padrão zero têm o mesmo valor em todas as linhas.
Estas colunas não têm qualquer variância. Significa simplesmente que não existe uma relação significativa com quaisquer outras colunas. Estas colunas não podem ajudar a diferenciar as classes ou grupos de dados. Por isso, eliminámos as colunas que não têm variação.

In [6]:
data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Attack Types
0,54865,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.0,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [7]:
data[data['Attack Types'] != 'BENIGN']

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Attack Types
18600,80,1293792,3,7,26,11607,20,0,8.666667,10.263203,...,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
18601,80,4421382,4,0,24,0,6,6,6.000000,0.000000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
18602,80,1083538,3,6,26,11601,20,0,8.666667,10.263203,...,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
18603,80,80034360,8,4,56,11601,20,0,7.000000,5.656854,...,20,939.0,0.0,939,939,39300000.0,44200000.0,70600000,8072664,DDoS
18604,80,642654,3,6,26,11607,20,0,8.666667,10.263203,...,20,0.0,0.0,0,0,0.0,0.0,0,0,DDoS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522275,80,11512204,8,5,326,11632,326,0,40.750000,115.258408,...,32,892.0,0.0,892,892,6507197.0,0.0,6507197,6507197,DoS
2522281,80,11513325,5,5,471,3525,471,0,94.199997,210.637604,...,32,918.0,0.0,918,918,6508582.0,0.0,6508582,6508582,DoS
2522317,80,11509201,7,6,314,11632,314,0,44.857143,118.680847,...,32,899.0,0.0,899,899,6503248.0,0.0,6503248,6503248,DoS
2522344,80,11509095,8,5,369,11632,369,0,46.125000,130.461197,...,32,914.0,0.0,914,914,6504954.0,0.0,6504954,6504954,DoS


### Aplicação de PCA para reduzir as dimensões

Uma forma simples e eficaz de reduzir a dimensionalidade do conjunto de dados e melhorar o desempenho do modelo é utilizar caraterísticas fortemente correlacionadas. Utilizámos a codificação de etiquetas na caraterística alvo, em que os valores numéricos atribuídos a cada categoria não têm significado inerente e são arbitrários. Por este motivo, a matriz de correlação calculada utilizando variáveis codificadas por rótulos pode não refletir com precisão as verdadeiras relações entre as variáveis.

Assim, uma abordagem mais flexível para a seleção de caraterísticas pode ser a PCA. A PCA é uma técnica que transforma o conjunto original de variáveis num conjunto mais pequeno de variáveis não correlacionadas, designadas por componentes principais.

A PCA pode captar relações mais complexas entre variáveis que podem não ser evidentes na análise da matriz de correlação. Também pode ajudar a reduzir o risco de sobreajuste.

Neste caso, aplicámos a ACP incremental. A ACP incremental é uma variante da ACP que permite o cálculo eficiente de componentes principais de um grande conjunto de dados que não pode ser armazenado na memória.

Aplicamos o StandardScaler antes de executar a ACP incremental para padronizar os valores de dados num formato padrão.

In [8]:
# Importando a classe StandardScaler da biblioteca sklearn.preprocessing
from sklearn.preprocessing import StandardScaler

# Separando as variáveis independentes (features) da variável dependente (target)
features = data.drop('Attack Types', axis = 1)  # 'Attack Type' é a coluna que queremos prever, então ela é retirada das features
attacks = data['Attack Types']  # Armazenamos a coluna 'Attack Type' separadamente como variável dependente (target)
# Inicializando o objeto StandardScaler para realizar a normalização
scaler = StandardScaler()  # O StandardScaler irá normalizar as variáveis numéricas para que tenham média 0 e desvio padrão 1
# Aplicando o scaler para normalizar as features
scaled_features = scaler.fit_transform(features)  # 'fit_transform' ajusta o scaler aos dados e depois os transforma

In [9]:
print(features.columns)

Index(['Destination Port', 'Flow Duration', 'Total Fwd Packets',
       'Total Backward Packets', 'Total Length of Fwd Packets',
       'Total Length of Bwd Packets', 'Fwd Packet Length Max',
       'Fwd Packet Length Min', 'Fwd Packet Length Mean',
       'Fwd Packet Length Std', 'Bwd Packet Length Max',
       'Bwd Packet Length Min', 'Bwd Packet Length Mean',
       'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s',
       'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
       'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max',
       'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std',
       'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags',
       'Fwd Header Length', 'Bwd Header Length', 'Fwd Packets/s',
       'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length',
       'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance',
       'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count',

In [10]:
# Importando a classe IncrementalPCA da biblioteca sklearn.decomposition
from sklearn.decomposition import IncrementalPCA

# Calculando o número de componentes principais que queremos manter
size = len(features.columns) // 2  # Estamos definindo o número de componentes principais como a metade do número de colunas em 'features'
# Inicializando o objeto IncrementalPCA com o número de componentes e o tamanho do lote (batch_size)
ipca = IncrementalPCA(n_components=size, batch_size=500)  # A opção batch_size define o tamanho de cada lote que será processado por vez
# Dividindo os dados em lotes (batches) para processamento incremental
for batch in np.array_split(scaled_features, len(features) // 500):
    ipca.partial_fit(batch)  # Para cada lote de dados, o método partial_fit ajusta o modelo IncrementalPCA
# Exibindo a proporção de variância explicada pelos componentes principais selecionados
print(f'information retained: {sum(ipca.explained_variance_ratio_):.2%}')

information retained: 99.23%


In [11]:
# Aplicando a transformação PCA para os dados normalizados
transformed_features = ipca.transform(scaled_features)  # Transformando as features usando os componentes principais calculados pelo IncrementalPCA

# Criando um novo DataFrame com as componentes principais como colunas
new_data = pd.DataFrame(transformed_features, columns=[f'PC{i+1}' for i in range(size)])  
# A cada componente principal gerado, nomeamos a coluna como 'PC1', 'PC2', ..., até 'PCn', onde n é o número de componentes

# Adicionando a coluna 'Attack Type' ao novo DataFrame com as componentes principais
new_data['Attack Types'] = attacks.values  # Incluímos a variável target ('Attack Type') para que possamos relacionar os componentes principais com o tipo de ataque

In [12]:
new_data

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,Attack Types
0,-2.311094,-0.052684,0.515875,0.616537,3.840727,0.395336,-0.017878,0.186557,0.370079,-0.680438,...,-0.233945,0.699887,-0.539784,-0.035279,0.023075,0.001733,0.045647,0.151301,0.051935,BENIGN
1,-2.246553,-0.049159,0.467881,0.395550,2.001551,-0.141045,-0.016487,-0.780967,-0.889976,2.660582,...,-0.012903,0.543839,0.785574,0.212940,0.030939,0.001218,0.025860,0.009198,-0.058324,BENIGN
2,-2.258822,-0.049501,0.473634,0.408672,2.081408,-0.132962,-0.016754,-0.769681,-0.877466,2.634068,...,-0.020362,0.539934,0.780933,0.203564,0.034717,0.001194,0.025898,0.005914,-0.064384,BENIGN
3,-2.249188,-0.050635,0.467051,0.346824,2.013841,-0.106530,-0.016178,-0.745133,-0.840229,2.506786,...,-0.027380,0.175826,0.802770,0.063856,0.047573,0.001131,0.009281,0.018965,-0.033301,BENIGN
4,-2.311090,-0.052685,0.515877,0.616515,3.840698,0.395327,-0.017879,0.186543,0.370100,-0.680454,...,-0.233960,0.699809,-0.539762,-0.035319,0.023078,0.001733,0.045645,0.151311,0.051946,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522357,-1.966993,-0.034823,0.214152,-0.550069,-0.715274,-0.255957,-0.028814,-0.395631,0.253441,-0.399620,...,-0.296905,-0.081193,0.269844,-0.117746,-0.025322,-0.000129,0.121039,-0.385284,-0.365042,BENIGN
2522358,-1.967205,-0.034920,0.140546,-0.769750,-1.329699,-1.050008,-0.084008,-0.676149,1.052839,-0.349890,...,0.177611,0.145811,0.015451,-0.222039,-0.507178,-0.003223,0.002393,-0.005697,-0.005829,BENIGN
2522359,-2.177556,-0.044307,0.882352,1.291548,4.455997,0.141248,-0.035232,0.677099,1.469905,-0.910787,...,-0.031176,1.551953,-0.121564,0.613116,0.076038,0.000554,0.116441,-0.021843,-0.250166,BENIGN
2522360,-1.906970,-0.035866,0.236786,-0.648835,-0.915041,-0.480387,-0.047541,-0.496969,0.535691,-0.471521,...,-0.168155,0.066397,0.364400,-0.251553,-0.355645,-0.001837,0.073125,-0.243405,-0.267952,BENIGN


In [13]:
# Salvando o novo dataset (com componentes principais) em um arquivo CSV
new_data.to_csv('datasets/CICIDS2017/processed_dataset.csv', index=False)